In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Sort the CSV you saved creaded in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Ancud,CL,2020-04-18 01:23:56,-41.87,-73.82,55.40,100,100,4.70,light rain,0.12,0.0
1,1,Saskylakh,RU,2020-04-18 01:23:56,71.92,114.08,2.48,88,8,5.28,clear sky,0.00,0.0
2,2,Kapaa,US,2020-04-18 01:20:55,22.08,-159.32,80.60,61,1,11.41,clear sky,0.00,0.0
3,3,Bambous Virieux,MU,2020-04-18 01:23:56,-20.34,57.76,75.20,78,40,3.36,scattered clouds,0.00,0.0
4,4,Busselton,AU,2020-04-18 01:23:57,-33.65,115.33,59.00,94,100,5.84,overcast clouds,0.00,0.0


In [12]:
# Ask the customer to add a minimum and maximum temperature value and precipitation preferences.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preference = input("Do you want it to be raining? (yes/no) ")
snow_preference = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [14]:
# Filter the dataset to find the cities that fit the criteria.
if rain_preference == "no" and snow_preference == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Max Temp"] <= max_temp) & \
                                          (city_data_df["Rain inches (last 3 hrs)"] == 0) & \
                                          (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif rain_preference == "yes" and snow_preference == "no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Max Temp"] <= max_temp) & \
                                          (city_data_df["Rain inches (last 3 hrs)"] > 0) & \
                                          (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif rain_preference == "no" and snow_preference == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Max Temp"] <= max_temp) & \
                                          (city_data_df["Rain inches (last 3 hrs)"] == 0) & \
                                          (city_data_df["Snow inches (last 3 hrs)"] > 0)]
elif rain_preference == "yes" and snow_preference == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Max Temp"] <= max_temp) & \
                                          (city_data_df["Rain inches (last 3 hrs)"] > 0) & \
                                          (city_data_df["Snow inches (last 3 hrs)"] > 0)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
2,2,Kapaa,US,2020-04-18 01:20:55,22.08,-159.32,80.60,61,1,11.41,clear sky,0.0,0.0
3,3,Bambous Virieux,MU,2020-04-18 01:23:56,-20.34,57.76,75.20,78,40,3.36,scattered clouds,0.0,0.0
6,6,Rikitea,PF,2020-04-18 01:23:58,-23.12,-134.97,77.23,64,31,1.34,scattered clouds,0.0,0.0
17,17,Isangel,VU,2020-04-18 01:24:01,-19.55,169.27,86.00,70,75,5.82,broken clouds,0.0,0.0
20,20,Avarua,CK,2020-04-18 01:24:02,-21.21,-159.78,82.40,61,64,6.93,broken clouds,0.0,0.0


In [15]:
# Create DataFrame called hotel_df to store hotel names alongside city, country, max temp, current description, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,80.60,clear sky,22.08,-159.32,
3,Bambous Virieux,MU,75.20,scattered clouds,-20.34,57.76,
6,Rikitea,PF,77.23,scattered clouds,-23.12,-134.97,
17,Isangel,VU,86.00,broken clouds,-19.55,169.27,
20,Avarua,CK,82.40,broken clouds,-21.21,-159.78,
23,Mahebourg,MU,75.20,scattered clouds,-20.41,57.70,
25,Georgetown,MY,84.20,few clouds,5.41,100.34,
33,San Andres,CO,82.40,few clouds,12.58,-81.70,
34,Ndele,CF,76.51,scattered clouds,8.41,20.65,
35,Hambantota,LK,77.00,few clouds,6.12,81.12,


In [16]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use base URL to search for hotels in Paris.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# Save the new Dataframe with hotel information based on customer preferences.
hotel_df.to_csv("weather_data/WeatherPy_vacation.csv", index_label="City_ID")
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,80.60,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
3,Bambous Virieux,MU,75.20,scattered clouds,-20.34,57.76,Casa Tia Villa
6,Rikitea,PF,77.23,scattered clouds,-23.12,-134.97,Pension Maro'i
17,Isangel,VU,86.00,broken clouds,-19.55,169.27,Volcano Sea View Tree House
20,Avarua,CK,82.40,broken clouds,-21.21,-159.78,Paradise Inn


In [19]:
# Create maker info box template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Create map with preferred hotel locations.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))